# 1. Download the pretrained model and upload to s3

In [1]:
!pip install Transformers

     |████████████████████████████████| 3.1 MB 20.6 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 100 kB/s  eta 0:00:01
     |████████████████████████████████| 895 kB 66.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 63.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
from transformers import BertForSequenceClassification

pretrained_model="bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(pretrained_model)
model.save_pretrained("./model")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [25]:
help(model.save_pretrained)
# dir(model)

Help on method save_pretrained in module transformers.modeling_utils:

save_pretrained(save_directory:Union[str, os.PathLike], save_config:bool=True, state_dict:Union[dict, NoneType]=None, save_function:Callable=<function save at 0x7f6ec9cf1b70>, push_to_hub:bool=False, **kwargs) method of transformers.models.bert.modeling_bert.BertForSequenceClassification instance
    Save a model and its configuration file to a directory, so that it can be re-loaded using the
    `:func:`~transformers.PreTrainedModel.from_pretrained`` class method.
    
    Arguments:
        save_directory (:obj:`str` or :obj:`os.PathLike`):
            Directory to which to save. Will be created if it doesn't exist.
        save_config (:obj:`bool`, `optional`, defaults to :obj:`True`):
            Whether or not to save the config of the model. Useful when in distributed training like TPUs and need
            to call this function on all processes. In this case, set :obj:`save_config=True` only on the main
     

In [4]:
!ls -rtlh ./model/
!cd model && tar czvf ../model.tar.gz *
!cd ../ && ls -lrt

total 418M
-rw-rw-r-- 1 ec2-user ec2-user  678 Dec  6 13:43 config.json
-rw-rw-r-- 1 ec2-user ec2-user 418M Dec  6 13:43 pytorch_model.bin
config.json
pytorch_model.bin
total 36
drwx------ 2 root     root     16384 Nov 29 03:17 lost+found
drwxrwxr-x 6 ec2-user ec2-user  4096 Nov 29 08:41 sagemaker_train_demo
drwxrwxr-x 4 ec2-user ec2-user  4096 Nov 29 08:42 docker_test_folder
-rw-rw-r-- 1 ec2-user ec2-user  1140 Nov 30 04:00 Untitled.ipynb
drwxrwxr-x 8 ec2-user ec2-user  4096 Dec  1 07:44 sagemaker-pytorch-inference-toolkit
drwxrwxr-x 7 ec2-user ec2-user  4096 Dec  6 13:44 amazon-sagemaker-bert-pytorch


In [5]:
!pwd

/home/ec2-user/SageMaker/amazon-sagemaker-bert-pytorch


In [1]:
import boto3
import os
import numpy as np
import pandas as pd
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/ivy-pretrained-pytorch-bert"

role = sagemaker.get_execution_role()
print(bucket)
print(role)

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
# boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
s3_pretrain_model = "s3://{}".format(os.path.join(bucket, key))
print(s3_pretrain_model)

sagemaker-us-east-1-833719260605
arn:aws:iam::833719260605:role/ivy-sagemaker-role
s3://sagemaker-us-east-1-833719260605/sagemaker/ivy-pretrained-pytorch-bert/model.tar.gz


In [22]:
!aws s3 cp s3://sagemaker-us-east-1-833719260605/grammer-classification-bert-base-uncase-2021-12-02-10-33-48-251/model.tar.gz trained_model.tar.gz

download: s3://sagemaker-us-east-1-833719260605/grammer-classification-bert-base-uncase-2021-12-02-10-33-48-251/model.tar.gz to ./trained_model.tar.gz


In [21]:
!aws s3 ls s3://sagemaker-us-east-1-833719260605/sagemaker/ivy-pretrained-pytorch-bert/model.tar.gz

2021-12-06 13:45:46  405255270 model.tar.gz


# 2. Prepare the Inference Script
To deploy a pretrained PyTorch model, we will need to use the PyTorch estimator object to create a PyTorchModel object and set a different entry_point.

An implementation of model_fn is required for inference script. We are going to use default implementations of input_fn, predict_fn, output_fn and model_fn

# 3. Deploy With the pretrained model and inference script

## 3.1 Create Model Object

In [2]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=s3_pretrain_model,
    role=role,
    entry_point='deploy_ei.py',
    source_dir='code',
    framework_version='1.3.1',
    py_version='py3',
    sagemaker_session=sagemaker_session
)

## 3.2 Create inference endpoint

In [3]:
import time

endpoint_name = 'grammar-classification-{}-ep'.format(time.time()).replace('.', '').replace('_', '')
instance_type = "ml.t2.medium" #'ml.m5.large'
# Function will exit before endpoint is finished creating
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
#     accelerator_type=accelerator_type,
    endpoint_name=endpoint_name,
    wait=True,
)


-----------!

In [4]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

res = predictor.predict('Please remember to delete me when you are done.')
print("Predicted class:", np.argmax(res, axis=1))

Predicted class: [0]


# 4. Test the endpoint
the code can run outside sagemaker, like lambda function

In [5]:
import boto3
sagemaker_runtime_client = boto3.Session().client(service_name='runtime.sagemaker')

In [6]:
import json
def predict_grammer_text(text):
    response = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType='application/json', 
        Body=json.dumps(text)
    )
    result = response['Body'].read()
    result = json.loads(result)
    print('Probabilities for all classes: ', result)
    predicted_class = np.argmax(result)
    if predicted_class == 0:
        print('Grammer incorrect!')
    else:
        print('Grammer correct.')
    return predicted_class

In [7]:
s = "I am a girl come from Chinese."
x = predict_grammer_text(s)
print(x)

Probabilities for all classes:  [[0.517944872379303, 0.37516987323760986]]
Grammer incorrect!
0


In [8]:
s = "I am a girl from China."
x = predict_grammer_text(s)
print(x)

Probabilities for all classes:  [[0.5746420621871948, 0.3318248689174652]]
Grammer incorrect!
0


# 5. Clean the endpoint
<b>Please remember to clean the endpoint if it is just for practice</b><br/>
the endpoint will create ec2 instance in the backend to serve the predict request. if we didn't clean the endpoint, the ec2 instance will keep running in the backend which will generate cost.

In [22]:
print(predictor.endpoint_name)
resp = sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)
print(type(resp), resp)

resp = sagemaker_session.delete_endpoint_config(endpoint_config_name=predictor.endpoint_name)
print(type(resp), resp)

grammar-classification-16388043152128417-ep
<class 'NoneType'> None


In [10]:
print(pytorch_model.name)
resp = sagemaker_session.delete_model(model_name=pytorch_model.name)
print(type(resp), resp)

pytorch-inference-2021-12-06-15-25-44-762
<class 'NoneType'> None
